In [1]:
! pip install -q transformers datasets peft accelerate bitsandbytes
! pip install setfit

In [2]:
import huggingface_hub
# Uncomment and run if you need to log in to Hugging Face
huggingface_hub.login()

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from setfit import SetFitModel, SetFitTrainer


classification_model = SetFitModel.from_pretrained(
    "aicoral048/setfit-fined-tuned-books",
)

ner_model = AutoModelForTokenClassification.from_pretrained("aicoral048/ner-books-model-final")


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

config_setfit.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/28.3k [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [4]:
import torch
from transformers import pipeline as hf_pipeline


# Initialize NER pipeline
ner_pipeline = hf_pipeline(
    "token-classification",
    model=ner_model,
    tokenizer=AutoTokenizer.from_pretrained("aicoral048/ner-books-model-final"),
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

def book_recommendation_pipeline(user_query):
    """
    Pipeline that takes a user query and extracts:
    - Possible genres using the classification model
    - Authors and titles using the NER model
    """
    print(f"User Query: {user_query}")
    print("-" * 50)

    # 1. Extract genres using classification model
    # SetFitModel.predict returns a list of binary lists (0/1 for each label)
    predicted_genres_binary = classification_model.predict([user_query])[0]  # e.g., [0, 0, 1, 0, 0, 0, 0, 1]

    # Hardcode genre names as provided
    genre_names = ['Classics', 'Contemporary', 'Fantasy', 'Historical Fiction', 'Mystery', 'Nonfiction', 'Romance', 'Young Adult']

    # Find indices where binary is 1
    predicted_genres_indices = [i for i, val in enumerate(predicted_genres_binary) if val == 1]

    # Map indices to genre names
    predicted_genre_names = [genre_names[i] for i in predicted_genres_indices]

    print(f"Predicted Genres: {predicted_genre_names}")

    # 2. Extract entities using NER model
    entities = ner_pipeline(user_query)
    authors = [ent['word'] for ent in entities if ent['entity_group'] == 'AUTHOR']
    titles = [ent['word'] for ent in entities if ent['entity_group'] == 'TITLE']

        # Filter out invalid titles (e.g., too long or containing query-like words)
    filtered_titles = []
    for title in titles:
        title_words = title.split()
        # Discard if longer than 5 words or contains words like "like", "similar", "recommend"
        if len(title_words) <= 5 and not any(word.lower() in ['like', 'similar', 'recommend', 'book', 'novel'] for word in title_words):
            filtered_titles.append(title)

    print(f"Extracted Authors: {authors}")
    print(f"Extracted Titles: {filtered_titles}")

    # Return structured output
    return {
        'query': user_query,
        'genres': predicted_genre_names,
        'authors': authors,
        'titles': filtered_titles
    }

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
# Example Query 1
result1 = book_recommendation_pipeline("Can you recommend a fantasy novel like Harry Potter by J.K. Rowling?")
print("Result:", result1)
print("\n")

User Query: Can you recommend a fantasy novel like Harry Potter by J.K. Rowling?
--------------------------------------------------
Predicted Genres: ['Fantasy']
Extracted Authors: ['J. K. Rowling']
Extracted Titles: ['Harry Potter']
Result: {'query': 'Can you recommend a fantasy novel like Harry Potter by J.K. Rowling?', 'genres': ['Fantasy'], 'authors': ['J. K. Rowling'], 'titles': ['Harry Potter']}




/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [7]:
# Example Query 2
result2 = book_recommendation_pipeline("I want a romance book similar to Pride and Prejudice.")
print("Result:", result2)
print("\n")

User Query: I want a romance book similar to Pride and Prejudice.
--------------------------------------------------
Predicted Genres: ['Historical Fiction', 'Romance']
Extracted Authors: []
Extracted Titles: ['Pride and Prejudice']
Result: {'query': 'I want a romance book similar to Pride and Prejudice.', 'genres': ['Historical Fiction', 'Romance'], 'authors': [], 'titles': ['Pride and Prejudice']}




/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [8]:
# Example Query 3
result3 = book_recommendation_pipeline("I love reading One Hundred Years of Solitude by Gabriel García Márquez.")
print("Result:", result3)
print("\n")

User Query: I love reading One Hundred Years of Solitude by Gabriel García Márquez.
--------------------------------------------------
Predicted Genres: ['Classics', 'Historical Fiction']
Extracted Authors: ['Gabriel García Márquez']
Extracted Titles: ['One Hundred Years of Solitude']
Result: {'query': 'I love reading One Hundred Years of Solitude by Gabriel García Márquez.', 'genres': ['Classics', 'Historical Fiction'], 'authors': ['Gabriel García Márquez'], 'titles': ['One Hundred Years of Solitude']}




/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
